In [1]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from tqdm import tqdm

### Set Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Hyperparameters

In [3]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

### Load Data

In [4]:
train_dataset = datasets.MNIST(root='dataset/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



### Create Bidirectional LSTM

In [5]:
class BRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear((hidden_size*2)*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    # Forward Prop
    out, _ = self.lstm(x, (h0, c0))
    out = out.reshape(out.shape[0], -1)
    # out = self.fc(out[:,-1,:]) # all training examples, last hidden state, all features
    out = self.fc(out)
    return out

### Initialize the Model

In [6]:
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

### Loss & Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Remember to give model parameters here

### Training the Model

In [8]:
for epoch in tqdm(range(num_epochs)):
  for ind, (data, target) in enumerate(train_loader):
    data = data.to(device=device).squeeze(1)
    target = target.to(device=device)
    
    # forward
    scores = model(data)
    loss = criterion(scores, target)

    # backward
    optimizer.zero_grad() # set all the gradients to 0 for each batch
    loss.backward()

    # gradient descent or adam step - updates the parameters
    optimizer.step()

  print(f"Epoch {epoch+1} is done")

 20%|██        | 1/5 [00:25<01:41, 25.48s/it]

Epoch 1 is done


 40%|████      | 2/5 [00:50<01:16, 25.37s/it]

Epoch 2 is done


 60%|██████    | 3/5 [01:16<00:50, 25.43s/it]

Epoch 3 is done


 80%|████████  | 4/5 [01:41<00:25, 25.45s/it]

Epoch 4 is done


100%|██████████| 5/5 [02:07<00:00, 25.41s/it]

Epoch 5 is done


### Testing the Model

In [9]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() # Let the model know that this is evaluation mode

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device).squeeze(1)
      y = y.to(device=device)

      scores = model(x)
      
      # Which class has the max value
      _, pred = scores.max(dim=1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0) # 64

    print(f"Got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples)*100):.2f}%")

In [10]:
check_accuracy(test_loader, model)

Got 9880 / 10000 with accuracy 98.80%
